In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv


In [2]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, callbacks, utils, optimizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import torch

2025-10-09 16:10:55.122404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760026255.473448      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760026255.570993      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [4]:
data_path = '/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv'
data = pd.read_csv(data_path)

# Aperçu
display(data.head())
print("\nInfo :")
print(data.info())
print("\nDistribution originale des labels :")
print(data['label'].value_counts())

,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."



Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526724 entries, 0 to 1526723
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   label   1526724 non-null  int64 
 1   text    1526724 non-null  object
dtypes: int64(1), object(1)
memory usage: 23.3+ MB
None

Distribution originale des labels :
label
0    771604
1    755120
Name: count, dtype: int64


In [5]:
TARGET_PER_CLASS = 600000
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

def safe_sample_per_class(df, label_col='label', n=TARGET_PER_CLASS, random_state=SEED):
    parts = []
    for lbl, group in df.groupby(label_col):
        sample_n = min(len(group), n)
        parts.append(group.sample(n=sample_n, random_state=random_state))
    return pd.concat(parts).reset_index(drop=True)

data_balanced = safe_sample_per_class(data, label_col='label', n=TARGET_PER_CLASS)
print("Nouvelle distribution (après échantillonnage sécurisé) :")
print(data_balanced['label'].value_counts())

Nouvelle distribution (après échantillonnage sécurisé) :
label
0    600000
1    600000
Name: count, dtype: int64


In [6]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\.\S+', ' ', text)           # URLs
    text = re.sub(r'@\w+', ' ', text)                      # mentions
    text = re.sub(r'#', ' ', text)                         # retirer le dièse (garder mot)
    text = re.sub(r'[^a-z0-9àâäéèêëïîôöùûüçœæ\-\s]', ' ', text)  # garder lettres FR basiques + chiffres + dash
    text = re.sub(r'\s+', ' ', text).strip()               # espaces multiples
    return text

# Appliquer (création d'une colonne 'text_clean')
data_balanced['text_clean'] = data_balanced['text'].astype(str).map(clean_text)
display(data_balanced[['text', 'text_clean']].head())

,text,text_clean
0,"Non, je ne la regarde pas, idk elle.",non je ne la regarde pas idk elle
1,"- Très, très vrai.",- très très vrai
2,Allongé au lit et essayant de dormir ne se pas...,allongé au lit et essayant de dormir ne se pas...
3,"S'endormir, rêver à propos de Tom et un amour ...",s endormir rêver à propos de tom et un amour q...
4,"Avoir eu des problèmes majeurs avec la santé, ...",avoir eu des problèmes majeurs avec la santé l...


In [7]:
import transformers
from transformers import AutoTokenizer, AutoModel

# Solution 1: Utiliser un modèle MiniLM plus récent et stable
MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
print("Chargement du tokenizer et du modèle MiniLM...")

# Chargement avec paramètres pour éviter les erreurs
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Vérification des spécifications du modèle
print(f"\nArchitecture du modèle MiniLM:")
print(f"- Couches: {model.config.num_hidden_layers}")
print(f"- Dimensions cachées: {model.config.hidden_size}")
print(f"- Têtes d'attention: {model.config.num_attention_heads}")

Chargement du tokenizer et du modèle MiniLM...


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]


Architecture du modèle MiniLM:
- Couches: 12
- Dimensions cachées: 384
- Têtes d'attention: 12


In [8]:
def mean_pooling(model_output, attention_mask):
    """
    Applique le mean pooling sur les token embeddings
    """
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_minilm_embeddings(texts, batch_size=32):
    """
    Extrait les embeddings de texte using le vrai modèle MiniLM
    """
    all_embeddings = []
    
    # Passage du modèle en mode évaluation
    model.eval()
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenization
        encoded_input = tokenizer(
            batch_texts.tolist(),
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        
        # Passage through le modèle MiniLM
        with torch.no_grad():
            model_output = model(**encoded_input)
        
        # Mean pooling pour obtenir l'embedding de la phrase
        embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        all_embeddings.extend(embeddings.numpy())
    
    return np.array(all_embeddings)

In [9]:
test_texts = [
    "J'adore ce produit, il est fantastique !",
    "Je déteste ce service, c'est horrible.",
    "Le film était moyen, ni bon ni mauvais."
]

print("Test des embeddings MiniLM sur des exemples français:")
test_embeddings = get_minilm_embeddings(pd.Series(test_texts))

for i, (text, embedding) in enumerate(zip(test_texts, test_embeddings)):
    print(f"\nTexte {i+1}: '{text}'")
    print(f"Embedding shape: {embedding.shape}")
    print(f"Valeurs min/max: {embedding.min():.4f}/{embedding.max():.4f}")

# Calcul des similarités cosinus
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(test_embeddings)
print(f"\nMatrice de similarité cosinus:")
print(similarity_matrix)

Test des embeddings MiniLM sur des exemples français:

Texte 1: 'J'adore ce produit, il est fantastique !'
Embedding shape: (384,)
Valeurs min/max: -0.7168/0.6489

Texte 2: 'Je déteste ce service, c'est horrible.'
Embedding shape: (384,)
Valeurs min/max: -0.9004/0.6716

Texte 3: 'Le film était moyen, ni bon ni mauvais.'
Embedding shape: (384,)
Valeurs min/max: -0.6038/0.6964

Matrice de similarité cosinus:
[[1.0000002  0.39389148 0.40278012]
 [0.39389148 1.0000002  0.26599473]
 [0.40278012 0.26599473 1.0000001 ]]
